# 数据准备

In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

/Users/luliang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
print('train images     :', mnist.train.images.shape,
      'labels:'           , mnist.train.labels.shape)
print('validation images:', mnist.validation.images.shape,
      ' labels:'          , mnist.validation.labels.shape)
print('test images      :', mnist.test.images.shape,
      'labels:'           , mnist.test.labels.shape)

train images     : (55000, 784) labels: (55000, 10)
validation images: (5000, 784)  labels: (5000, 10)
test images      : (10000, 784) labels: (10000, 10)


# 建立模型

In [3]:
def layer(output_dim,input_dim,inputs, activation=None):
    W = tf.Variable(tf.random_normal([input_dim, output_dim]))
    b = tf.Variable(tf.random_normal([1, output_dim]))
    XWb = tf.matmul(inputs, W) + b
    if activation is None:
        outputs = XWb
    else:
        outputs = activation(XWb)
    return outputs

In [4]:
# 建立输入层 x 

In [5]:
x = tf.placeholder("float", [None, 784])

In [6]:
# 建立隐藏层h1

In [7]:
h1=layer(output_dim=1000,input_dim=784,
         inputs=x ,activation=tf.nn.relu)  


In [8]:
# 建立隐藏层h2

In [9]:
h2=layer(output_dim=1000,input_dim=1000,
         inputs=h1 ,activation=tf.nn.relu)  

In [10]:
# 建立输出层

In [11]:
y_predict=layer(output_dim=10,input_dim=1000,
                inputs=h2,activation=None)

In [12]:
#建立训练数据label真实值 placeholder

In [13]:
y_label = tf.placeholder("float", [None, 10])

# 定义训练方式

In [14]:
#定义loss function

In [15]:
loss_function = tf.reduce_mean(
                   tf.nn.softmax_cross_entropy_with_logits
                       (logits=y_predict , 
                        labels=y_label))

In [16]:
#选择optimizer

In [17]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001) \
                    .minimize(loss_function)

# 定义评估模型的准确率

In [18]:
#计算每一项数据是否正确预测

In [19]:
correct_prediction = tf.equal(tf.argmax(y_label  , 1),
                              tf.argmax(y_predict, 1))

In [20]:
#将计算预测正确结果，加总平均

In [21]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# 开始训练

In [ ]:
trainEpochs = 15
batchSize = 100
totalBatchs = int(mnist.train.num_examples/batchSize)
epoch_list=[];accuracy_list=[];loss_list=[];
from time import time
startTime=time()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
for epoch in range(trainEpochs):

    
    for i in range(totalBatchs):
        batch_x, batch_y = mnist.train.next_batch(batchSize)
        sess.run(optimizer,feed_dict={x: batch_x,
                                      y_label: batch_y})
        
    
    loss,acc = sess.run([loss_function,accuracy],
                        feed_dict={x: mnist.validation.images, 
                                   y_label: mnist.validation.labels})

    epoch_list.append(epoch)
    loss_list.append(loss);accuracy_list.append(acc)    
    
    print("Train Epoch:", '%02d' % (epoch+1), \
          "Loss=","{:.9f}".format(loss)," Accuracy=",acc)
    
duration =time()-startTime
print("Train Finished takes:",duration)        

Train Epoch: 01 Loss= 134.655517578  Accuracy= 0.9186
Train Epoch: 02 Loss= 96.240570068  Accuracy= 0.9326
Train Epoch: 03 Loss= 74.054565430  Accuracy= 0.9448
Train Epoch: 04 Loss= 63.637401581  Accuracy= 0.9536
Train Epoch: 05 Loss= 62.615085602  Accuracy= 0.9542
Train Epoch: 06 Loss= 58.139911652  Accuracy= 0.9584
Train Epoch: 07 Loss= 60.881088257  Accuracy= 0.9594
Train Epoch: 08 Loss= 55.440376282  Accuracy= 0.9564
Train Epoch: 09 Loss= 53.998626709  Accuracy= 0.9632
Train Epoch: 10 Loss= 60.341800690  Accuracy= 0.9608
Train Epoch: 11 Loss= 62.641826630  Accuracy= 0.9612
Train Epoch: 12 Loss= 51.310192108  Accuracy= 0.9648
Train Epoch: 13 Loss= 54.407024384  Accuracy= 0.9674
Train Epoch: 14 Loss= 55.606781006  Accuracy= 0.966


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.gcf()
fig.set_size_inches(4,2)
plt.plot(epoch_list, loss_list, label = 'loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss'], loc='upper left')

In [ ]:
plt.plot(epoch_list, accuracy_list,label="accuracy" )
fig = plt.gcf()
fig.set_size_inches(4,2)
plt.ylim(0.8,1)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()
plt.show()

# 评估模型的准确率

In [ ]:
print("Accuracy:", sess.run(accuracy,
                           feed_dict={x: mnist.test.images, 
                                      y_label: mnist.test.labels}))

# 进行预测

In [ ]:
prediction_result=sess.run(tf.argmax(y_predict,1),
                           feed_dict={x: mnist.test.images })


In [ ]:
prediction_result[:10]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def plot_images_labels_prediction(images,labels,
                                  prediction,idx,num=10):
    fig = plt.gcf()
    fig.set_size_inches(12, 14)
    if num>25: num=25 
    for i in range(0, num):
        ax=plt.subplot(5,5, 1+i)
        
        ax.imshow(np.reshape(images[idx],(28, 28)), 
                  cmap='binary')
            
        title= "label=" +str(np.argmax(labels[idx]))
        if len(prediction)>0:
            title+=",predict="+str(prediction[idx]) 
            
        ax.set_title(title,fontsize=10) 
        ax.set_xticks([]);ax.set_yticks([])        
        idx+=1 
    plt.show()

In [ ]:
plot_images_labels_prediction(mnist.test.images,
                              mnist.test.labels,
                              prediction_result,0)

In [ ]:
y_predict_Onehot=sess.run(y_predict,
                          feed_dict={x: mnist.test.images })

In [ ]:
y_predict_Onehot[8]

# 找出预测错误

In [ ]:
for i in range(400):
    if prediction_result[i]!=np.argmax(mnist.test.labels[i]):
        print("i="+str(i)+"   label=",np.argmax(mnist.test.labels[i]),
              "predict=",prediction_result[i])

In [ ]:
#sess.close()